In [5]:
import numpy as np
import cv2 as cv
import os
k = 60
mstd_features = np.zeros((len(os.listdir('imgs'))*(360//k)*(480//k),6))

for i in range(len(os.listdir('imgs'))):
    img = cv.imread('imgs/'+os.listdir('imgs')[i])
    
    colors_sum = img[...,0] + img[...,1] + img[...,2]
    r_ratio = img[...,2]/(colors_sum+0.0001)
    g_ratio = img[...,1]/(colors_sum+0.0001)
    norm = colors_sum/3

    for j in range(img.shape[0]//k):
        for l in range(img.shape[1]//k):
            r_avg = []
            g_avg = []
            norm_avg = []
            
            r_std = []
            g_std = []
            norm_std = []
            r_avg.append(np.mean(r_ratio[j*k:(j+1)*k,l*k:(l+1)*k]))
            g_avg.append(np.mean(g_ratio[j*k:(j+1)*k,l*k:(l+1)*k]))
            norm_avg.append(np.mean(norm[j*k:(j+1)*k,l*k:(l+1)*k]))

            r_std.append(np.std(r_ratio[j*k:(j+1)*k,l*k:(l+1)*k]))
            g_std.append(np.std(g_ratio[j*k:(j+1)*k,l*k:(l+1)*k]))
            norm_std.append(np.std(norm[j*k:(j+1)*k,l*k:(l+1)*k]))

            # reshape the arrays to 1D
            r_avg = np.array(r_avg).reshape(-1)
            g_avg = np.array(g_avg).reshape(-1)
            norm_avg = np.array(norm_avg).reshape(-1)
            r_std = np.array(r_std).reshape(-1)
            g_std = np.array(g_std).reshape(-1)
            norm_std = np.array(norm_std).reshape(-1)
            # hstack the arrays 
            mstd = (np.hstack((r_avg, g_avg, norm_avg, r_std, g_std, norm_std))) 
            mstd_features[i] = mstd
   

In [7]:
def distance(x,y):
    return np.sqrt(np.sum((x-y)**2))

def mean(x):
    return np.sum(x,axis=0)/len(x)

def k_means_clustering(X,k):
    # centers = X[np.random.choice(k,X.shape[1],replace=False),:]
    # initializing centers to 0
    centers = np.zeros((k,X.shape[1]))

    # randomly initializing centers
    for i in range(k):
        centers[i] = X[np.random.randint(0,X.shape[0]),:]

    labels = np.zeros(X.shape[0])
    
    dist = np.zeros((X.shape[0],k))
 
    n_iter = 0
    
    n_changes = 0

    prev_labels = centers.copy()
    while True:
        for i in range(k):
            # dist[:,i] = np.apply_along_axis(distance,1,X,centers[i,:])
            dist[:, i] = np.sqrt(np.sum((X - centers[i, :]) ** 2, axis=1))
        labels = np.argmin(dist,axis=1)
       
        for i in range(k):
            if(len(X[labels==i])>0):
                centers[i,:] = mean(X[labels==i,:])
  
        n_changes = len(centers[np.abs(centers-prev_labels)>0.0001*np.ones(centers.shape)])
    
        prev_labels = centers.copy()
       
        n_iter += 1
  
        if n_changes==0:
            break
    return labels,centers,n_iter

In [8]:
labels,centers,n_iter = k_means_clustering(mstd_features,30)

In [9]:
with open('image_words_mstd.txt','w') as f:
    for i in range(centers.shape[0]):
        f.write(str(centers[i,:])+'\n')